In [38]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda, Compose
import matplotlib.pyplot as plt
from torch import nn, optim
import numpy as np
import os

In [2]:
# Download training data from open datasets.
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

# Download test data from open datasets.
test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

/Users/noon/PycharmProjects/jamesnick/venv/lib/python3.8/site-packages/torchvision/datasets/mnist.py:498: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  ../torch/csrc/utils/tensor_numpy.cpp:180.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [3]:
batch_size = 64

# Create data loaders.
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

def make_acc_dataset(model, dataset):
  n = len(dataset)
  for i in range(n):
      x, y = dataset[i][0], dataset[i][1]
      with torch.no_grad():
          pred = model(x)
      if (not(pred[0].argmax(0) - y)):
          dataset.targets[i] = 1
      else:
          dataset.targets[i] = 0
  return dataset

In [4]:
# As we save
device = "cpu"
print("Using {} device".format(device))
# Define model
class NeuralNetworkClass(nn.Module):
    def __init__(self):
        super(NeuralNetworkClass, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
            nn.ReLU()
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x).to(device)
        return logits

class_model = NeuralNetworkClass().to(device)
print(class_model)

class NeuralNetworkPred(nn.Module):
    def __init__(self):
        super(NeuralNetworkPred, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 2),
            nn.ReLU()
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x).to(device)
        return logits
pred_model = NeuralNetworkPred().to(device)
print(pred_model)

Using cpu device
NeuralNetworkClass(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
    (5): ReLU()
  )
)
NeuralNetworkPred(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=2, bias=True)
    (5): ReLU()
  )
)


In [6]:
cwd = os.getcwd()
path = os.getcwd()
model = NeuralNetworkPred()
model.load_state_dict(torch.load(path + os.path.sep + '1' + '_pred_model.pth'))

<All keys matched successfully>

In [32]:
# Lets make a function that loads the models into a python list
def load_acc_models(path,n_of_models):
    model_list = []
    for i in range(n_of_models):
        new_model = NeuralNetworkPred()
        new_model.load_state_dict(torch.load(path + os.path.sep + str(i + 1) + '_pred_model.pth'))
        new_model.eval()
        model_list.append(new_model)
        del new_model
    return model_list
        
# Lets make a function that loads the models into a python list
def load_class_models(path,n_of_models):
    model_list = []
    for i in range(n_of_models):
        new_model = NeuralNetworkClass()
        new_model.load_state_dict(torch.load(path + os.path.sep + str(i + 1) + '_allclass_model.pth'))
        new_model.eval()
        model_list.append(new_model)
        del new_model
    return model_list
    

In [68]:
# 
path = os.getcwd()
n_of_models = 10

list_of_acc_model = load_acc_models(path,n_of_models)
list_of_class_model = load_class_models(path,n_of_models)

In [72]:
# Load in one data image and then evaluate it on the best model for acc..

x = test_data[0][0]

def get_acc_model_predict_list(x,model_list):
    predict_list = np.zeros(len(model_list))
    for i, model in enumerate(model_list):
        with torch.no_grad():
            pred = model(x)
        predict_list[i] = pred[0,1].numpy()
    return predict_list

pred_list = get_acc_model_predict_list(x,list_of_acc_model)

In [35]:
# Test an small chuck on the predicted vs. actual

x = test_data[0][0]
y = test_data[0][1]
def get_class_model_predict_list(x,y,model_list):
    predict_list = []
    for model in model_list:
        with torch.no_grad():
            pred = model(x)
        print(pred[0].argmax(0))
        predict_list.append(pred)
    return predict_list

class_list = get_class_model_predict_list(x,y,list_of_class_model)

tensor(9)
tensor(1)
tensor(1)
tensor(1)
tensor(1)
tensor(1)
tensor(1)
tensor(1)
tensor(1)
tensor(1)


In [78]:
# Lets test the final model.

def evaluate_hydra(data,acc_model_lists,class_model_list):
    correct = 0
    total = 0
    for i, d in enumerate(data):
        predict_list = get_acc_model_predict_list(d[0],acc_model_lists)
        with torch.no_grad():
            pred = class_model_list[predict_list.argmax()](d[0])
            _, predicted = torch.max(pred.data, 1)
            label = d[1]
            print(f'Predicted class = {predicted}, actual = {label}')
            correct += (predicted == d[1]).sum().item()
        if i == 20:
            break
    return len(data), correct


total, correct = evaluate_hydra(test_data,list_of_acc_model,list_of_class_model)


print('Accuracy of the network on the 10000 test images: %d %%' % (
    100 * correct / total))

Predicted class = tensor([9]), actual = 9
Predicted class = tensor([2]), actual = 2
Predicted class = tensor([1]), actual = 1
Predicted class = tensor([1]), actual = 1
Predicted class = tensor([6]), actual = 6
Predicted class = tensor([1]), actual = 1
Predicted class = tensor([4]), actual = 4
Predicted class = tensor([4]), actual = 6
Predicted class = tensor([1]), actual = 5
Predicted class = tensor([1]), actual = 7
Predicted class = tensor([2]), actual = 4
Predicted class = tensor([5]), actual = 5
Predicted class = tensor([5]), actual = 7
Predicted class = tensor([1]), actual = 3
Predicted class = tensor([4]), actual = 4
Predicted class = tensor([1]), actual = 1
Predicted class = tensor([2]), actual = 2
Predicted class = tensor([2]), actual = 4
Predicted class = tensor([8]), actual = 8
Predicted class = tensor([0]), actual = 0
Predicted class = tensor([0]), actual = 2
Accuracy of the network on the 10000 test images: 0 %
